<img src="https://ithaka-labs.s3.amazonaws.com/static-files/images/tdm/tdmdocs/CC_BY.png" align=left alt="CC BY license logo" /><br /><br />
Created by [Nathan Kelber](http://nkelber.com) and Ted Lawless for [JSTOR Labs](https://labs.jstor.org/) under [Creative Commons CC BY License](https://creativecommons.org/licenses/by/4.0/)<br />
**For questions/comments/improvements, email nathan.kelber@ithaka.org.**<br />
____

# Exploring Metadata and Pre-Processing

**Description of methods in this notebook:**
This [notebook](https://constellate.org/docs/key-terms/#jupyter-notebook) shows how to explore and pre-process the [metadata](https://constellate.org/docs/key-terms/#metadata) of a [dataset](https://constellate.org/docs/key-terms/#dataset) using [Pandas](https://constellate.org/docs/key-terms/#pandas). 

The following processes are described:

* Importing a [CSV file](https://constellate.org/docs/key-terms/#csv-file) containing the [metadata](https://constellate.org/docs/key-terms/#metadata) for a given dataset ID
* Creating a [Pandas](https://constellate.org/docs/key-terms/#pandas) dataframe to view the [metadata](https://constellate.org/docs/key-terms/#metadata)
* Pre-processing your [dataset](https://constellate.org/docs/key-terms/#dataset) by filtering out unwanted texts
* Importing the full [JSON Lines (.jsonl)](https://constellate.org/docs/key-terms/#jsonl) dataset and filtering it
* Visualizing the metadata of your pre-processed [dataset](https://constellate.org/docs/key-terms/#dataset) by the number of documents/year and pages/year

**Use Case:** For Learners (Detailed explanation, not ideal for researchers)

**Difficulty:** Intermediate

**Completion time:** 60 minutes

**Knowledge Required:** 
* Python Basics Series ([Start Python Basics 1](../Python-basics/python-basics-1.ipynb))

**Knowledge Recommended:**

* Pandas Series ([Start Pandas 1](../Pandas-basics/pandas-basics-1.ipynb))

**Data Format:** 
* [CSV file](https://constellate.org/docs/key-terms/#csv-file)
* [JSON Lines (.jsonl)](https://constellate.org/docs/key-terms/#jsonl)

**Libraries Used:**
* [constellate](https://constellate.org/docs/key-terms/#constellate-client) client to retrieve the [metadata](https://constellate.org/docs/key-terms/#metadata) in a [CSV file](https://constellate.org/docs/key-terms/#csv-file)
* [pandas](https://constellate.org/docs/key-terms/#pandas) to manipulate and visualize the metadata

**Research Pipeline:** None
____

In [ ]:
### Import modules and libraries ###

import constellate
import pandas as pd
import os
import json
from pathlib import Path
import gzip

# For displaying plots inline with Jupyter Notebooks
%matplotlib inline

## Import your dataset metadata file (.csv)

The next code cell tries to import your dataset metadata with each of the following methods:

1. Search for a custom dataset metadata file in the data folder 
2. Download a full dataset metadata file that has been requested
3. Download a sampled dataset metadata file (1500 items) that builds automatically when a dataset is created

If you are using a [dataset ID](https://constellate.org/docs/key-terms/#dataset-ID), replace the default dataset ID in the next code cell.

If you don't have a dataset ID, you can:
* Use the sample dataset ID already in the code cell
* [Create a new dataset](https://constellate.org/builder)
* [Use a dataset ID from other pre-built sample datasets](https://constellate.org/dashboard/datasets)

The Constellate client will download dataset metadata automatically using either the `.download()` or `.get_metadata()` method.
* Full dataset metadata files are downloaded using the `.download()` method. They must be requested first in the builder environment. See the [Constellate client documentation](https://constellate.org/docs/constellate-client).

* Sampled dataset metadata files (1500 items) are downloaded using the `.get_metadata()` method. They are built automatically when a dataset is created.

In [ ]:
dataset_id = "7e41317e-740f-e86a-4729-20dab492e925"

# Check if a data folder exists. If not, create it.
data_folder = Path('./data/')
data_folder.mkdir(exist_ok=True)

# Check to see if a dataset file exists
# If not, download a dataset using the Constellate Client
# The default dataset is Shakespeare Quarterly, 1950-present
dataset_metadata = Path.cwd() / 'data' / 'my_metadata.csv' # Make sure this filepath matches your dataset metadata filename

if dataset_metadata.exists() == False:
    try: 
        dataset_metadata = constellate.download(dataset_id, 'metadata')
        print(f'Full dataset metadata ready')
    except:
        dataset_metadata = constellate.get_metadata(dataset_id)
        print(f'Sampled dataset metadata ready')

We are ready to import pandas for our analysis and create a dataframe. We will use the `read_csv()` method to create our dataframe from the CSV file.

In [ ]:
# Create our dataframe using Pandas
df = pd.read_csv(dataset_metadata)

In [ ]:
df

We can confirm the size of our dataset using the `len()` function on our dataframe.

In [ ]:
original_document_count = len(df)
print(f'Total original documents: {original_document_count}')

Now let's take a look at the data in our dataframe `df`. We will set pandas to show all columns using `set_option()` then get a preview using `head()`.

In [ ]:
# Set the pandas option to show all columns
# Setting None gives us all columns
# To show less columns replace None with an integer
pd.set_option('display.max_columns', None) 

# Set maximumum number of rows to 50
pd.set_option('display.max_rows', 50)

# Show the first five rows of our dataframe
# To show a different number of preview rows
# Pass an integer into the .head()
df.head() 

Here are descriptions for the metadata types found in each column:

|Column Name|Description|
|---|---|
|id|a unique item ID (In JSTOR, this is a stable URL)|
|title|the title for the item|
|isPartOf|the larger work that holds this title (for example, a journal title)|
|publicationYear|the year of publication|
|doi|the digital object identifier for an item|
|docType|the type of document (for example, article or book)|
|provider|the source or provider of the dataset|
|datePublished|the publication date in yyyy-mm-dd format|
|issueNumber|the issue number for a journal publication|
|volumeNumber|the volume number for a journal publication|
|url|a URL for the item and/or the item's metadata|
|creator|the author or authors of the item|
|publisher|the publisher for the item|
|language|the language or languages of the item (eng is the ISO 639 code for English)|
|pageStart|the first page number of the print version|
|pageEnd|the last page number of the print version|
|placeOfPublication|the city of the publisher|
|wordCount|the number of words in the item|
|pageCount|the number of print pages in the item|
|outputFormat|what data is available ([unigrams](https://constellate.org/docs/key-terms/#unigram), [bigrams](https://constellate.org/docs/key-terms/#bigram), [trigrams](https://constellate.org/docs/key-terms/#trigram), and/or full-text)|

___

## Filtering out columns using Pandas

If there are any columns you would like to drop from your analysis, you can drop them with:

`df = df.drop(['column_name1', 'column_name2', ...], axis=1)`

In [ ]:
# Drop each of these named columns
# axis=1 specifies we are dropping columns
# axis=0 would specify to drop rows
df = df.drop(['outputFormat', 'pageEnd', 'pageStart', 'datePublished'], axis=1)

# Show the first five rows of our updated dataframe
df.head()

## Filtering out rows with Pandas

Now that we have filtered out unwanted metadata columns, we can begin filtering out any texts that may not match our research interests. Let's examine the first and last ten rows of the dataframe to see if we can identify texts that we would like to remove. We are looking for patterns in the metadata that could help us remove many texts at once.

In [ ]:
# Preview the first ten items in the dataframe
# Can you identify patterns to select rows to remove?
df.head(10)

In [ ]:
# Preview the last ten items in the dataframe
# Can you identify patterns to select rows to remove?
df.tail(10)

In [ ]:
# We create a function to report how many documents were removed.

def texts_report(pre_count):
    """Prints out a report of:
    1. How many documents were removed
    2. The total original number of documents
    3. The total current number of documents
    """
    
    removed_count = pre_count - len(df)
    print(f'{removed_count} texts were removed.')
    print(f'Total original documents: {original_document_count}')
    print('Total current documents: ', len(df))

### Remove all rows without data for a particular column

For example, we may wish to remove any texts that do not have authors. (In the case of journals, this may be helpful for removing paratextual sections such as the table of contents, indices, etc.) The column of interest in this case is `creator`. 

In [ ]:
# Remove all texts without an author

print('Removing texts without authors...')
initial_count = len(df)
df = df.dropna(subset=['creator']) #drop each row that has no value under 'creators'

# Report the number of texts removed
texts_report(initial_count)


### Remove row based on the content of a particular column

We can also remove texts, depending on whether we do (or do not) want a particular value in a column. Here are a few examples.

In [ ]:
# Remove all items with a particular title
# Change title to desired column
# Change `Review Article` to your undesired title
title_to_remove = 'Review Article'

# Removing texts
print(f'Removing texts with title "{title_to_remove}"...')
initial_count = len(df)

# Create a filter that returns all titles that do not match `title_to_remove`
# Apply the filter to the DataFrame
title_filter = df['title'] != title_to_remove
df = df[title_filter]

# Report the number of texts removed
texts_report(initial_count)

In [ ]:
# Keep only items with a particular language
# Change language to desired column
# Change 'eng' to your desired language
language = 'eng' # Change to another language code for other languages

# Removing texts
print(f'Removing texts not in "{language}" language...')
initial_count = len(df)

# Create a filter that returns all languages matching `language` variable
language_filter = df['language'] == language
df = df[language_filter] # Apply filter to the DataFrame

# Report the number of texts removed
texts_report(initial_count)

In [ ]:
# Remove all items with less than 1500 words
# Change `min_word_count to your desired minimum number of words
min_word_count = 1500

# Removing texts
print(f'Removing texts with fewer than {min_word_count} words...')
initial_count = len(df)

# Create a filter that 
word_count_filter = df['wordCount'] > min_word_count
df = df[word_count_filter]

# Report the number of texts removed
texts_report(initial_count)

Take a final look at your dataframe to make sure the current texts fit your research goals. In the next step, we will save the IDs of your pre-processed dataset.

In [ ]:
# Preview the first 50 rows of your dataset
# If all the items look good, move to the next step.
len(df)

## Filtering the Full JSON-L Dataset

We created our filtered list of IDs from the metadata CSV. Next, we will download the full dataset (JSON-L file) and then filter out any unwanted documents. 

In [ ]:
# Create a Python list of IDs we want from the Pandas dataframe
filtered_id_list = df["id"].tolist()

We can now download the sampled (1500 items) dataset (or the full dataset if it has been requested). See more:
* [Constellate dataset types](https://constellate.org/docs/what-format-are-jstor-portico-datasets)
* [Dataset options](https://constellate.org/docs/dataset-options)
* [Constellate client](https://constellate.org/docs/constellate-client)


In [ ]:
# Pull in the sampled (1500 items) dataset JSON using
# The .get_dataset() method downloads the sampled JSON file
dataset_file = constellate.get_dataset(dataset_id)

# Download the full dataset (JSON-L file)
# The full dataset must be requested first
#dataset_file = constellate.download(dataset_id, 'jsonl')

Next, we will create an empty JSON-L file to filter our results into. We will also check if we want to overwrite the file, if it already exists.

In [ ]:
### Create the filtered JSON-L file ###
# This may take several minutes to complete


# Check if a data folder exists. If not, create it.
data_folder = Path('./data/')
data_folder.mkdir(exist_ok=True)

# Define the file output name
file_path = Path.cwd() / 'data' / 'my_data.jsonl' # You may change the name of the file here

# Delete output files if they already exist
if file_path.exists():
    overwrite = input(f'Overwrite {file_path}? (yes/no)')
    if overwrite.lower() == 'yes':
        print(f'Overwriting older version of {file_path}')
        file_path.unlink()
        file_path.touch()

We can now filter through each document in the original JSON file. All documents that match our filtered ID list will be added to the new JSON file. 

In [ ]:
# Append all documents with ids in `filtered_id_list`
for document in constellate.dataset_reader(dataset_file):
    document_id = document['id']
    # Append any documents in the filtered list
    if document_id in filtered_id_list:
        with file_path.open('a') as outfile:
            json.dump(document, outfile)
            outfile.write('\n')
print(f'{file_path} created.')

Finally, we compress the dataset file using `gzip`. The compression may take some time, but it will make it easier to transfer the file. 

In [ ]:
# Compress the file using gzip
# This may take several minutes to complete
f_in = file_path.open('rb')
f_out = gzip.open(f'{file_path}.gz', 'wb')
f_out.writelines(f_in)
f_out.close()
f_in.close()
print(f'Compression complete. \n{file_path}.gz has been created.')

<h3 style="color:red; display:inline">Note! &lt; / &gt; </h3>

It is a good idea to keep a copy of your dataset in your lab and on your local machine as backup. The Constellate lab has a storage limit of 30 GB.

---
## Visualizing the Pre-Processed Data

In [ ]:
# Group the data by publication year and the aggregated number of ids into a bar chart
df.groupby(['publicationYear'])['id'].agg('count').plot.bar(title='Documents by year', figsize=(20, 5), fontsize=12); 

# Read more about Pandas dataframe plotting here: 
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.plot.html

And now let's look at the total page numbers by year.

In [ ]:
# Group the data by publication year and aggregated sum of the page counts into a bar chart

df.groupby(['publicationYear'])['pageCount'].agg('sum').plot.bar(title='Pages by decade', figsize=(20, 5), fontsize=12);